In [42]:
import os 
import json

In [43]:
def get_ba_graph_paths(dataset_dir= "./datasets/"):
    graph_list = []
    for dirpath, _, files in os.walk(dataset_dir):
        for filename in files:
            try:
                if filename.startswith("ba_edgelist") and filename.endswith(".edges"):
                    file_path = os.path.join(dirpath, filename) 
                    graph_list.append((file_path, os.path.splitext(filename)[0]))
            except Exception as e: 
                print(e, f'{filename}')
    return graph_list

In [44]:
import networkx as nx
import pandas as pd

def get_graph_paths(dataset_dir= "./datasets/"):
    graph_list = []
    for dirpath, _, files in os.walk(dataset_dir):
        for filename in files:
            try:
                if filename.endswith(".edges"):
                    file_path = os.path.join(dirpath, filename) 
                    graph_list.append((file_path, os.path.splitext(filename)[0]))
            except Exception as e: 
                print(e, f'{filename}')
    return graph_list


def get_sir_graph_paths(net_name, num_b=3,  result_path = './datasets/SIR_Results/'):
    paths= []
    for i in range(num_b):
        sir_dir =os.path.join(result_path, net_name)
        sir_dir = os.path.join(sir_dir, f'{i}.csv')
        paths.append(sir_dir)
    return paths


def get_previously_sim_values(sir_graph_path):
    try:
        df = pd.read_csv(sir_graph_path)
        values = df['Node'].tolist()
        return values
    except OSError as e:
        return []

def getTotalNumNodes(net_name):
    graph_list = get_graph_paths()
    path = ''
    for tmp in graph_list:
        if tmp[1]==net_name:
            path = tmp[0]
            
    G = nx.read_edgelist(path, comments="%", nodetype=int)
    return G.number_of_nodes()


def getSimNumNodes(net_name):
    sir_paths = get_sir_graph_paths(net_name)
    temp = [  len(get_previously_sim_values(path)) for path in sir_paths ]
    return (temp)


def getSortValue(net_edges_path):
    net_edges_path=net_edges_path[0]
    G = nx.read_edgelist(net_edges_path, comments="%", nodetype=int)
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    avg_degree = 2 * num_edges / num_nodes if num_nodes > 0 else 0
    return num_nodes+avg_degree

def getSubList(graphs, index, step):
    sublits = []
    for i in range(index, len(graphs), step):
        sublits.append(graphs[i])
    return sublits

In [45]:
temp = get_ba_graph_paths()
print(temp)
list.sort(temp, key=getSortValue)
print(temp)
ba_mch_0 = getSubList(temp, 0, 3)
ba_mch_1 = getSubList(temp, 1, 3)
ba_mch_2 = getSubList(temp, 2, 3)
print(len(ba_mch_0))
print(len(ba_mch_1))
print(len(ba_mch_2))

[('./datasets/BA\\ba_edgelist_1000_10.edges', 'ba_edgelist_1000_10'), ('./datasets/BA\\ba_edgelist_1000_20.edges', 'ba_edgelist_1000_20'), ('./datasets/BA\\ba_edgelist_1000_4.edges', 'ba_edgelist_1000_4'), ('./datasets/BA\\ba_edgelist_2000_10.edges', 'ba_edgelist_2000_10'), ('./datasets/BA\\ba_edgelist_2000_20.edges', 'ba_edgelist_2000_20'), ('./datasets/BA\\ba_edgelist_2000_4.edges', 'ba_edgelist_2000_4'), ('./datasets/BA\\ba_edgelist_3000_10.edges', 'ba_edgelist_3000_10'), ('./datasets/BA\\ba_edgelist_3000_20.edges', 'ba_edgelist_3000_20'), ('./datasets/BA\\ba_edgelist_3000_4.edges', 'ba_edgelist_3000_4'), ('./datasets/BA_EXP\\ba_edgelist_exp1_3000_4.edges', 'ba_edgelist_exp1_3000_4'), ('./datasets/BA_EXP\\ba_edgelist_exp1_4000_4.edges', 'ba_edgelist_exp1_4000_4'), ('./datasets/BA_EXP\\ba_edgelist_exp1_5000_4.edges', 'ba_edgelist_exp1_5000_4'), ('./datasets/BA_EXP\\ba_edgelist_exp1_6000_4.edges', 'ba_edgelist_exp1_6000_4'), ('./datasets/BA_EXP\\ba_edgelist_exp1_7000_4.edges', 'ba_edg

## sort files



In [46]:

di = {}
di['BA_mch_0'] = [item[1] for item in ba_mch_0]
di['BA_mch_0'].append('ChicagoRegional')
di['BA_mch_0'].append('jazz')
di['BA_mch_0'].append('ia-crime-moreno')
di['BA_mch_0'].append('p2p-Gnutella04')

di['BA_mch_1'] = [item[1] for item in ba_mch_1]
di['BA_mch_1'].append('arenas-pgp')
di['BA_mch_1'].append('LastFM')
di['BA_mch_1'].append('sex')

di['BA_mch_2'] = [item[1] for item in ba_mch_2]
di['negin_mch'] = ['CA-GrQc', 'CA-HepTh', 'faa', 'facebook_combined', 'figeys', 'email', 'NS', 'Peh_edge', 'Stelzl', 'tvshow_edges', 'vidal', 'web-EPA']
di['mhd_mch'] = [ 'maybe-PROTEINS-full']
with open('machine.json', 'w') as f:
    json.dump(di, f)

In [47]:

notsorted = [] 
noteq = []
completed = []
incomplete = []
temp = False
for mch in di.keys():
    print(mch, ": ")
    i = 0

    t_=0
    s_=0
    for net in di[mch]:
        temp = False
        i+=1
        simNodes = getSimNumNodes(net)
        totalNumNodes = getTotalNumNodes(net)
        t_+=totalNumNodes
        s_+=simNodes[0]
        sortd = sorted(get_previously_sim_values(get_sir_graph_paths(net)[0])) ==get_previously_sim_values(get_sir_graph_paths(net)[0])
        if simNodes[0] >= totalNumNodes:
            temp = True
            completed.append(net)
        else: 
            incomplete.append(net)
        if not sortd :
            notsorted.append(net)
            print('### ', i, ".", net, "  process:", simNodes, '/',  totalNumNodes, 'completed: ',temp,  ' sorted:', sortd, '### ')
        elif not( simNodes[0]==simNodes[1] and simNodes[1]==simNodes[2]) :
            noteq.append(net)
            print('$$$ ', i, ".", net, "  process:", simNodes, '/',  totalNumNodes, 'completed: ',temp,  ' sorted:', sortd, ('$$$ '))
        else:
            print( i, ".", net, "  process:", simNodes, '/',  totalNumNodes, 'completed: ',temp,  ' sorted:', sortd)
    print("SIM NUM NODES / TOTAL NUM NODES: ",s_, ' / ', t_)
    print("percentage: ",(s_/ t_)*100)
    print("COMPLETED: ", completed)
    print("INCOMPLETED: ", incomplete)

    completed=[]
    incomplete = []
    print("-----------------")


BA_mch_0 : 
1 . ba_edgelist_1000_4   process: [1000, 1000, 1000] / 1000 completed:  True  sorted: True
2 . ba_edgelist_2000_4   process: [2000, 2000, 2000] / 2000 completed:  True  sorted: True
###  3 . ba_edgelist_exp2_2000_10   process: [2009, 2009, 2009] / 2000 completed:  True  sorted: False ### 
4 . ba_edgelist_3000_4   process: [3004, 3004, 3004] / 3000 completed:  True  sorted: True
5 . ba_edgelist_3000_20   process: [419, 419, 419] / 3000 completed:  False  sorted: True
###  6 . ba_edgelist_exp3_4000_6   process: [4023, 4023, 4023] / 4000 completed:  True  sorted: False ### 
###  7 . ba_edgelist_exp1_6000_4   process: [6021, 6021, 6021] / 6000 completed:  True  sorted: False ### 
8 . ChicagoRegional   process: [0, 0, 0] / 12979 completed:  False  sorted: True
9 . jazz   process: [0, 0, 0] / 198 completed:  False  sorted: True
10 . ia-crime-moreno   process: [0, 0, 0] / 829 completed:  False  sorted: True
11 . p2p-Gnutella04   process: [0, 0, 0] / 10876 completed:  False  sorted

### Example Output:
BA_mch_0 : 
1. ba_edgelist_1000_4   process: [0, 0, 0] / 1000  sorted: True
2. ba_edgelist_2000_4   process: [0, 0, 0] / 2000  sorted: True
3. ba_edgelist_exp2_2000_10   process: [481, 481, 481] / 2000  sorted: True
4. ba_edgelist_3000_4   process: [315, 315, 315] / 3000  sorted: True
5. ba_edgelist_3000_20   process: [0, 0, 0] / 3000  sorted: True
6. ba_edgelist_exp3_4000_6   process: [913, 913, 913] / 4000  sorted: True
7. ba_edgelist_exp1_6000_4   process: [604, 604, 604] / 6000  sorted: True


##### SIM NUM NODES/TOTAL NUM NODES:  2313  /  21000
##### percentage:  11.014285714285714
##### COMPLETED:  []
-----------------
BA_mch_1 : 
1. ba_edgelist_1000_10   process: [488, 488, 488] / 1000  sorted: True
2. ba_edgelist_exp2_2000_4   process: [0, 0, 0] / 2000  sorted: True
3. ba_edgelist_2000_20   process: [0, 0, 0] / 2000  sorted: True
4. ba_edgelist_exp1_3000_4   process: [320, 320, 320] / 3000  sorted: True
5. ba_edgelist_exp3_4000_2   process: [906, 906, 906] / 4000  sorted: True
6. ba_edgelist_exp3_4000_10   process: [241, 241, 241] / 4000  sorted: True
7. ba_edgelist_exp1_7000_4   process: [138, 138, 138] / 7000  sorted: True


##### SIM NUM NODES / TOTAL NUM NODES:  2093  /  23000
##### percentage:  9.1
##### COMPLETED:  []
-----------------
BA_mch_2 : 
1. ba_edgelist_1000_20   process: [0, 0, 0] / 1000  sorted: True
2. ba_edgelist_2000_10   process: [0, 0, 0] / 2000  sorted: True
3. ba_edgelist_exp2_2000_20   process: [500, 500, 500] / 2000  sorted: True
4. ba_edgelist_3000_10   process: [161, 161, 161] / 3000  sorted: True
5. ba_edgelist_exp1_4000_4   process: [249, 249, 249] / 4000  sorted: True
6. ba_edgelist_exp1_5000_4   process: [741, 741, 741] / 5000  sorted: True
7. ba_edgelist_exp1_8000_4   process: [122, 122, 122] / 8000  sorted: True

##### SIM NUM NODES / TOTAL NUM NODES:  1773  /  25000
##### percentage:  7.092
##### COMPLETED:  []
-----------------
negin_mch : 
1. CA-GrQc   process: [1015, 1015, 1015] / 5242  sorted: True
2. CA-HepTh   process: [411, 411, 411] / 9877  sorted: True
3. faa   process: [1226, 1226, 1226] / 1226  sorted: True
4. facebook_combined   process: [1460, 1460, 1460] / 4039  sorted: True
5. figeys   process: [1946, 1946, 1946] / 2239  sorted: True
6. email   process: [1133, 1133, 1133] / 1133  sorted: True
7. NS   process: [1461, 1461, 1461] / 1461  sorted: True
8. Peh_edge   process: [967, 967, 967] / 2426  sorted: True
9. Stelzl   process: [980, 980, 980] / 1706  sorted: True
10. tvshow_edges   process: [0, 0, 0] / 3892  sorted: True
11. vidal   process: [0, 0, 0] / 3133  sorted: True
12. web-EPA   process: [0, 0, 0] / 4271  sorted: True

##### SIM NUM NODES / TOTAL NUM NODES:  10599  /  40645
##### percentage:  26.0770082420962
##### COMPLETED:  ['faa', 'email', 'NS']
-----------------
mhd_mch : 
1. ChicagoRegional   process: [1385, 1385, 1385] / 12979  sorted: True
2. jazz   process: [198, 198, 198] / 198  sorted: True
3. ia-crime-moreno   process: [829, 829, 829] / 829  sorted: True
4. arenas-pgp   process: [1675, 1675, 1675] / 10680  sorted: True
5. LastFM   process: [1859, 1859, 1859] / 7624  sorted: True
6. maybe-PROTEINS-full   process: [322, 322, 322] / 43466  sorted: True
7. p2p-Gnutella04   process: [1134, 1134, 1134] / 10876  sorted: True
8. sex   process: [0, 0, 0] / 10106  sorted: True
   
##### SIM NUM NODES / TOTAL NUM NODES:  7402  /  96758
##### percentage:  7.650013435581554
##### COMPLETED:  ['jazz', 'ia-crime-moreno']
-----------------


## Sort and save

In [48]:
run = False


In [49]:
if run:
    for net in notsorted:
        sir_paths = get_sir_graph_paths(net)
        i = 0
        for path in sir_paths:
            i+=1
            df = pd.read_csv(path)
            df_sorted = df.sort_values(by='Node')
            df_sorted.to_csv(path, index=False)